In [1]:
import logging
import os
import shutil
import dill as pickle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from streamline.runners.auto_runner import AutoRunner
from streamline.runners.clean_optimize_runner import OptimizeClean


/Users/gabrielketron/AutoTest/AutoTestEnv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("./data/DigenData/all_new_test_sets.pkl",'rb') as f:
    digen = pickle.load(f)

#print(digen.keys())

digenkeys = list(digen.keys())

#print(digenkeys)
#print(digen['digen8_4426'])
digennoise = {}

In [3]:
for i in digenkeys:
    print(i) 
    data = pd.DataFrame(digen[i]['X'])
    yval = pd.DataFrame(digen[i]['y'])
    current_noise = digen[i]['noise']
    yval.columns = ['y']
    yval.rename(columns = {'0':'y'}, inplace = True)
    out = pd.concat([Test, yval], axis=1)
    out.index.name = 'InstanceId'
    digennoise[i] = current_noise
    if current_noise == 0.0: 
        outstring = './data/DigenStudy/No_Noise/' + i + '.csv'
        tempstring = './data/testfolder/' + i + '.csv'
        out.to_csv(outstring)
        out.to_csv(tempstring)
    else:
        outstring = './data/DigenStudy/Noisy/' + i + '.csv'
        tempstring = './data/testfolder/' + i + '.csv'
        out.to_csv(outstring)
        out.to_csv(tempstring)
    
    X_train, X_test,y_train, y_test = train_test_split(data, yval,test_size=0.08, shuffle=True)

    train_set = pd.concat([X_train, y_train], axis =1)
    train_set.index.name = 'InstanceId'
    tempstring = './data/testfolder/' + i +'_train.csv'
    train_set.to_csv(tempstring)
    cleanrun = OptimizeClean(dataset_name=[i], n_trials=100, data_path='./data/testfolder')
    cleanrun.run()
    optimal = cleanrun.param
    current_auc = cleanrun.goal

    tempstring = './data/testfolder/' + i +'_train.csv'
    test_set = pd.concat([X_test, y_test], axis =1)
    test_set.index.name = 'InstanceId'

    try:
        most_recent_run = AutoRunner(**optimal, dataset_names=[i],data_path='./data/testfolder',
                                        gen_report=False, clean=False, class_label='target', 
                                        instance_label='InstanceID', ml_algorithms=["NB", "LR", "DT", "EN"], 
                                        exclude=["GB", "RF","ANN","KNN","GP", 'eLCS', 'XCS', "ExSTraCS"]) # "XGB", "LGB", "CGB", "SVM","GB", "RF"
        output_csv = most_recent_run.run(run_para=False)
        performance = pd.read_csv(output_csv)
        summary_chart = performance
        goal = performance["ROC AUC"].max()
        best_model = performance.loc[performance["ROC AUC"].idxmax()][0]
        png_out = output_csv.removesuffix('Summary_performance_mean.csv')
        png_out = png_out + 'Summary_ROC.png'
        final_model_comparison = plt.show(png_out)
    except:
        print('EXCEPTION')
        
    print(current_auc-goal)
        


    



digen8_4426


type: name 'Test' is not defined

In [ ]:
#Noisy checks:



type: 'float' object is not subscriptable